In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 6

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
imgs_dir = r"C:\Users\cocka\Downloads\img_data\TrashType_Image_Dataset"

train_generator = train_datagen.flow_from_directory(
    imgs_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    imgs_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

base_model = tf.keras.applications.MobileNet(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


In [ ]:
model.save('waste_segregation_model.h5')

In [ ]:
savedModel = tf.keras.models.load_model('waste_segregation_model.h5')

In [ ]:
model = tf.keras.models.load_model('waste_segregation_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('model-phone.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
img = tf.keras.preprocessing.image.load_img('glass_464.jpg')
interpreter = tf.lite.Interpreter(model_path="model-phone.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']
input_data = np.array(img.resize((input_shape[1], input_shape[2])))  # Resize image to match input shape
input_data = (input_data.astype(np.float32) - 127.5) / 127.5  # Normalize if required

In [ ]:
output_details

In [ ]:
input_details

In [ ]:
# Set the input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'],np.expand_dims(input_data,axis=0))
output_data